<a href="https://colab.research.google.com/github/Bisrath1/amharic-ecommerce-ner-extractor/blob/main/NER.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install transformers datasets seqeval
!pip install torch


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 2.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16162 sha256=0c4523aae23c1103eb068c5b03b3d41c3ec1bf6bce78485142589c5ed04f9890
  Stored in directory: /root/.cache/pip/wheels/bc/92/f0/243288f899c2eacdfa8c5f9aede4c71a9bad0ee26a01dc5ead
Successfully built seqeval
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 123.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 96.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 65.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [18]:
import numpy as np

In [3]:
from google.colab import files
uploaded = files.upload()

Saving amharic_ner.conll to amharic_ner.conll


In [4]:
def load_conll(file_path):
    sentences = []
    sentence = []
    with open(file_path, 'r', encoding='utf-8') as f:  # Use the provided path
        for line in f:
            line = line.strip()
            if line:  # Non-empty line
                token, label = line.split()  # Split into token and label
                sentence.append((token, label))
            else:  # Empty line = end of sentence
                if sentence:
                    sentences.append(sentence)
                    sentence = []
        if sentence:  # Add the last sentence
            sentences.append(sentence)
    return sentences

# Load the dataset (use the correct path)
dataset = load_conll("amharic_ner.conll")

# Print first sentence
print(dataset[0])

[('ቡና', 'B-Product'), ('ማፍያ', 'I-Product'), ('ማሽን', 'I-Product'), ('ዋጋ', 'O'), ('ብር', 'O'), ('ውስን', 'O'), ('ፍሬ', 'O'), ('ነው', 'O'), ('የቀረው', 'O'), ('ጥራት', 'O'), ('ዋስትና', 'O'), ('ቅናሽ', 'O'), ('አድራሻ', 'O'), ('ቁጥር', 'O'), ('መገናኛ', 'B-LOC'), ('ዘፍመሽ', 'I-LOC'), ('ግራንድ', 'I-LOC'), ('ሞል', 'I-LOC'), ('ተኛ', 'O'), ('ፎቅ', 'O'), ('ቁጥር', 'O'), ('ቁጥር', 'O'), ('ጀሞ', 'B-LOC'), ('ከለላ', 'O'), ('ህንፃ', 'O'), ('ግራውንድ', 'O'), ('ለይ', 'O'), ('ማሳሰብያ', 'O'), ('ሱቃችን', 'O'), ('ሲመጡ', 'O'), ('ትክክለኛ', 'O'), ('የኛ', 'O'), ('ሱቅ', 'O'), ('መሆኑ', 'O'), ('ያረጋግጡ', 'O'), ('የራሳችን', 'O'), ('ሎጎ', 'O'), ('መኖሩን', 'O'), ('እና', 'O'), ('መሆኑ', 'O'), ('ያረጋግጡ', 'O'), ('ስልክ', 'O')]


In [5]:
from sklearn.model_selection import train_test_split

train_data, val_data = train_test_split(dataset, test_size=0.2, random_state=42)
print(f"Training sentences: {len(train_data)}, Validation sentences: {len(val_data)}")

Training sentences: 150, Validation sentences: 38


In [6]:
from datasets import Dataset

def convert_to_dict(data):
    return {
        'tokens': [[token for token, _ in sentence] for sentence in data],
        'ner_tags': [[label for _, label in sentence] for sentence in data]
    }

train_dict = convert_to_dict(train_data)
val_dict = convert_to_dict(val_data)

train_dataset = Dataset.from_dict(train_dict)
val_dataset = Dataset.from_dict(val_dict)

In [7]:
from transformers import AutoTokenizer, AutoModelForTokenClassification

model_name = "Davlan/afro-xlmr-base"  # Or "xlm-roberta-base"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForTokenClassification.from_pretrained(model_name, num_labels=7)  # 7 labels: B-Product, I-Product, B-LOC, I-LOC, B-PRICE, I-PRICE, O

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/398 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.08M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/707 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.11G [00:00<?, ?B/s]

Some weights of XLMRobertaForTokenClassification were not initialized from the model checkpoint at Davlan/afro-xlmr-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [8]:
label_list = ['O', 'B-Product', 'I-Product', 'B-LOC', 'I-LOC', 'B-PRICE', 'I-PRICE']
label2id = {label: idx for idx, label in enumerate(label_list)}
id2label = {idx: label for idx, label in enumerate(label_list)}

# Update model config
model.config.label2id = label2id
model.config.id2label = id2label

In [9]:
def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(
        examples['tokens'],
        truncation=True,
        is_split_into_words=True,
        padding=True,
        return_tensors="pt"
    )

    labels = []
    for i, label in enumerate(examples['ner_tags']):
        word_ids = tokenized_inputs.word_ids(batch_index=i)  # Map tokens to words
        label_ids = []
        previous_word_idx = None
        for word_idx in word_ids:
            if word_idx is None:  # Special tokens ([CLS], [SEP], [PAD])
                label_ids.append(-100)
            elif word_idx != previous_word_idx:  # First subword of a word
                label_ids.append(label2id[label[word_idx]])
            else:  # Subword continuation
                label_ids.append(label2id[label[word_idx]] if label[word_idx].startswith('I-') else -100)
            previous_word_idx = word_idx
        labels.append(label_ids)
    tokenized_inputs['labels'] = labels
    return tokenized_inputs

# Tokenize datasets
tokenized_train = train_dataset.map(tokenize_and_align_labels, batched=True)
tokenized_val = val_dataset.map(tokenize_and_align_labels, batched=True)

Map:   0%|          | 0/150 [00:00<?, ? examples/s]

Map:   0%|          | 0/38 [00:00<?, ? examples/s]

In [11]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir='./results',
    eval_strategy='epoch',  # Use 'eval_strategy' instead of 'evaluation_strategy'
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
    save_strategy='epoch',
    load_best_model_at_end=True,
)

In [12]:
from datasets import load_metric

metric = load_metric("seqeval")

def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    true_labels = [[label_list[l] for l in label if l != -100] for label in labels]
    true_predictions = [
        [label_list[p] for (p, l) in zip(pred, label) if l != -100]
        for pred, label in zip(predictions, labels)
    ]

    results = metric.compute(predictions=true_predictions, references=true_labels)
    return {
        'precision': results['overall_precision'],
        'recall': results['overall_recall'],
        'f1': results['overall_f1'],
        'accuracy': results['overall_accuracy'],
    }

/tmp/ipython-input-12-1952102776.py:3: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric("seqeval")


In [15]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir='./results',
    eval_strategy='epoch',  # Changed from evaluation_strategy
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
    save_strategy='epoch',
    load_best_model_at_end=True,
)

In [19]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_val,
    processing_class=tokenizer,  # Changed from tokenizer to processing_class
    compute_metrics=compute_metrics,
)

# Start training
trainer.train()

Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,No log,0.134312,0.000000,0.000000,0.000000,0.962906
2,No log,0.055700,1.000000,0.972973,0.986301,0.995363
3,No log,0.041971,1.000000,0.972973,0.986301,0.995363


/usr/local/lib/python3.11/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.11/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.11/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


TrainOutput(global_step=30, training_loss=0.13393274943033853, metrics={'train_runtime': 483.754, 'train_samples_per_second': 0.93, 'train_steps_per_second': 0.062, 'total_flos': 42947492276700.0, 'train_loss': 0.13393274943033853, 'epoch': 3.0})

In [20]:
eval_results = trainer.evaluate()
print(eval_results)

{'eval_loss': 0.04197053983807564, 'eval_precision': 1.0, 'eval_recall': 0.972972972972973, 'eval_f1': 0.9863013698630138, 'eval_accuracy': 0.9953632148377125, 'eval_runtime': 0.5333, 'eval_samples_per_second': 71.252, 'eval_steps_per_second': 5.625, 'epoch': 3.0}


/usr/local/lib/python3.11/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [21]:
model.save_pretrained('./fine_tuned_ner_model')
tokenizer.save_pretrained('./fine_tuned_ner_model')

('./fine_tuned_ner_model/tokenizer_config.json',
 './fine_tuned_ner_model/special_tokens_map.json',
 './fine_tuned_ner_model/sentencepiece.bpe.model',
 './fine_tuned_ner_model/added_tokens.json',
 './fine_tuned_ner_model/tokenizer.json')

In [22]:
from transformers import pipeline

ner_pipeline = pipeline('ner', model='./fine_tuned_ner_model', tokenizer='./fine_tuned_ner_model')

# Example Amharic sentence (replace with actual Amharic text)
sample_text = "ቦሌ አዲስ አበባ ውስጥ የሕፃን ጠርሙስ በ1000 ብር እንሸጣለን"
results = ner_pipeline(sample_text)

for entity in results:
    print(f"Entity: {entity['word']}, Label: {entity['entity']}, Score: {entity['score']:.4f}")

Device set to use cuda:0
